# Data cleaning and dataset creation

## Imports

In [ ]:
# External packages
import numpy as np
import pandas as pd
import sys as sys
from sklearn.model_selection import train_test_split

# Internal functions
from clean import load_data
